In [1]:
import mne
import pathlib
from mne.externals.pymatreader import read_mat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import axes3d

In [2]:
from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)

In [3]:
import os
os.chdir(r'/usr/slurm/venkatesh/HBN/')
subjs = sorted(os.listdir())[1:-2]
#cd


In [4]:
cd

/homes/v20subra


In [13]:
def csv_to_raw_mne(path_to_file,path_to_montage_ses,fs,path_to_events,filename,montage = 'GSN-HydroCel-129'):
    ''' Load csv files of data, chan locations and events and return a raw mne instance'''
    data = np.loadtxt(path_to_file,delimiter =',')
    chans = pd.read_csv(path_to_montage_ses,sep = ',',header = None)
    ch_list=['E1', 'E8', 'E14', 'E17', 'E21', 'E25', 'E32', 'E38', 'E43', 'E44', 'E48', 'E49', 'E56', 'E57', 'E63', 'E64', 'E69', 'E73', 'E74', 'E81', 'E82', 'E88', 'E89', 'E94', 'E95', 'E99', 'E100', 'E107', 'E113', 'E114', 'E119', 'E120', 'E121', 'E125', 'E126', 'E127', 'E128']
    ch_names = list(chans.values[1:,0])
#print(type(ch_names))
    #ch_names_appended = list(np.append(ch_names,'stim_channel'))
    #print(len(data[0]))
    types = ['eeg']*(len(ch_names))
    #types.append('stim')
    #data2 = np.zeros([1,len(data[0])]) #len(raw.times)
    #data_appended = np.append(data,data2,axis = 0)

    #print(np.shape(data_appended))
#print(len(types))

#types
    info = mne.create_info(ch_names,sfreq = fs,ch_types = types)
#raw=mne.io.RawArray(data, info)

#mne.find_events(raw,stim_channel='stim')
    raw = mne.io.RawArray(data, info)
    
    # set standard montage
    if montage:
        raw.set_montage(montage)

    # events array shape must be (n_events,3)The first column specifies the sample number of each event,
    # the second column is ignored, and the third column provides the event value.
    # If events already exist in the Raw instance at the given sample numbers, the event values will be added together.

    if path_to_events:
        # parse events file
        raw_events = pd.read_csv(path_to_events, sep = r'\s*,\s*', header = None, engine = 'python')
        values = raw_events[0].to_list()
        
        # identify markers start and stop recording
        idx = [i for i, e in enumerate(values) if e == 'break cnt']
        
         
        if filename == 'NDARDX770PJK':
           
            values.extend(["break cnt"])
            
            idx = [i for i, e in enumerate(values) if e == 'break cnt']
        
        samples = raw_events[1][1:idx[0]].to_numpy(dtype = int)
        # slicing until '-1' means that we will not know about the last state. Hence removed.
        event_values = raw_events[0][1:idx[0]].to_numpy(dtype = int)

        
        # append a last value for end of paradigm
        ## I think 1 acts as an explicit EOF, but having this slicing until '-1' as indicated
        # in the previous comment would not let us know the last state
        # event_values = np.append(event_values, 1)

        # Creating an array of len(samples)-1 would not have the sufficient length to add the 
        # sample's last row.
        events = np.zeros((len(samples), 3))
        
        events = events.astype('int')
        events[:, 0] = samples
        events[:, 2] = event_values
        
        # Appending one row of 'ones'. Will be easier to stop parsing once we hit 1
        events_final = np.append(events,np.ones((1, 3)),axis = 0).astype('int')
        raw = exclude_channels_from_raw(raw, ch_list)
        
    return raw,events_final

def exclude_channels_from_raw(raw,ch_to_exclude):
    '''Return a raw structure where ch_to_exclude are removed'''
    idx_keep = mne.pick_channels(raw.ch_names,include = raw.ch_names,exclude = ch_to_exclude)
    raw.pick_channels([raw.ch_names[pick] for pick in idx_keep])
    return raw



def preparation_resting_state(filename):
    path_to_file = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/RestingState_data.csv'% filename
    path_to_events = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/RestingState_event.csv' %filename
    path_to_montage_glob = '/S4B2/GSN_HydroCel_129_hbn.sfp'
    path_to_montage_ses = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/RestingState_chanlocs.csv' %filename
    fs = 500
    chans_glob = mne.channels.read_custom_montage(fname = 'S4B2/GSN_HydroCel_129_hbn.sfp') # read_montage is deprecated
# channels to exclude because noisy (Nentwich paper)


    raw, events = csv_to_raw_mne(path_to_file,path_to_montage_ses,fs,path_to_events,filename,montage = 'GSN-HydroCel-129')
    #raw.add_events(events, stim_channel = 'stim_channel',replace = False)
    return raw,events

def preparation(filename):
    path_to_file = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/Video3_data.csv'% filename
    path_to_events = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/Video3_event.csv' %filename
    path_to_montage_glob = '/S4B2/GSN_HydroCel_129_hbn.sfp'
    path_to_montage_ses = '/usr/slurm/venkatesh/HBN/%s/EEG/preprocessed/csv_format/Video3_chanlocs.csv' %filename
    fs = 500
    chans_glob = mne.channels.read_custom_montage(fname = 'S4B2/GSN_HydroCel_129_hbn.sfp') # read_montage is deprecated
# channels to exclude because noisy (Nentwich paper)


    raw, events = csv_to_raw_mne(path_to_file,path_to_montage_ses,fs,path_to_events,filename,montage = 'GSN-HydroCel-129')
    #raw.add_events(events, stim_channel = 'stim_channel',replace = False)
    return raw,events


a,b=preparation_resting_state('task1/NDARBF805EHN')

Creating RawArray with float64 data, n_channels=111, n_times=176386
    Range : 0 ... 176385 =      0.000 ...   352.770 secs
Ready.


In [14]:
sub1_raw,sub1_events = preparation(subjs[0])
sub2_raw,sub2_events = preparation(subjs[1])
sub3_raw,sub3_events = preparation(subjs[2])
sub4_raw,sub4_events = preparation(subjs[3])
sub5_raw,sub5_events = preparation(subjs[4])
sub6_raw,sub6_events = preparation(subjs[5])
sub7_raw,sub7_events = preparation(subjs[6])
sub8_raw,sub8_events = preparation(subjs[7])
sub9_raw,sub9_events = preparation(subjs[8])
sub10_raw,sub10_events = preparation(subjs[9])

Creating RawArray with float64 data, n_channels=111, n_times=86040
    Range : 0 ... 86039 =      0.000 ...   172.078 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=87720
    Range : 0 ... 87719 =      0.000 ...   175.438 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86030
    Range : 0 ... 86029 =      0.000 ...   172.058 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86092
    Range : 0 ... 86091 =      0.000 ...   172.182 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86564
    Range : 0 ... 86563 =      0.000 ...   173.126 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86045
    Range : 0 ... 86044 =      0.000 ...   172.088 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86058
    Range : 0 ... 86057 =      0.000 ...   172.114 secs
Ready.
Creating RawArray with float64 data, n_channels=111, n_times=86046
    Range : 0 ... 86045

In [6]:
def epochs(subject_raw,subject_events):

    epochs = mne.Epochs(subject_raw, subject_events, [83,103,9999], tmin=0, tmax=170,preload=True,baseline=(0,None))
    epochs_resampled = epochs#.resample(250) # Downsampling to 250Hz
    
    return epochs_resampled['83']



In [7]:
import numpy as np
from scipy.linalg import eigh
from timeit import default_timer


def train_cca(data):
    """Run Correlated Component Analysis on your training data.
        Parameters:
        ----------
        data : dict
            Dictionary with keys are names of conditions and values are numpy
            arrays structured like (subjects, channels, samples).
            The number of channels must be the same between all conditions!
        Returns:
        -------
        W : np.array
            Columns are spatial filters. They are sorted in descending order, it means that first column-vector maximize
            correlation the most.
        ISC : np.array
            Inter-subject correlation sorted in descending order
    """

    start = default_timer()

    C = len(data.keys())
    print(f'train_cca - calculations started. There are {C} conditions')

    gamma = 0.1
    Rw, Rb = 0, 0
    for cond in data.values():
        N, D, T, = cond.shape
        print(f'Condition has {N} subjects, {D} sensors and {T} samples')
        cond = cond.reshape(D * N, T)

        # Rij
        Rij = np.swapaxes(np.reshape(np.cov(cond), (N, D, N, D)), 1, 2)
        
        # Rw
        Rw = Rw + np.mean([Rij[i, i, :, :]
                           for i in range(0, N)], axis=0)
        
        # Rb
        Rb = Rb + np.mean([Rij[i, j, :, :]
                           for i in range(0, N)
                           for j in range(0, N) if i != j], axis=0)
        covariance = np.cov(cond)
    # Divide by number of condition
    Rw, Rb = Rw/C, Rb/C

    # Regularization
    Rw_reg = (1 - gamma) * Rw + gamma * np.mean(eigh(Rw)[0]) * np.identity(Rw.shape[0])

    # ISCs and Ws
    [ISC, W] = eigh(Rb, Rw_reg) #Eigen values, W matrix
    
    # Make descending order
    ISC, W = ISC[::-1], W[:, ::-1] 
    #print(ISC[0])
    stop = default_timer()

    print(f'Elapsed time: {round(stop - start)} seconds.')
    return W, ISC


def apply_cca(X, W, fs):
    """Applying precomputed spatial filters to your data.
        Parameters:
        ----------
        X : ndarray
            3-D numpy array structured like (subject, channel, sample)
        W : ndarray
            Spatial filters.
        fs : int
            Frequency sampling.
        Returns:
        -------
        ISC : ndarray
            Inter-subject correlations values are sorted in descending order.
        ISC_persecond : ndarray
            Inter-subject correlations values per second where first row is the most correlated.
        ISC_bysubject : ndarray
            Description goes here.
        A : ndarray
            Scalp projections of ISC.
    """

    start = default_timer()
    print('apply_cca - calculations started')

    N, D, T = X.shape
    # gamma = 0.1
    window_sec = 5
    X = X.reshape(D * N, T)
    
    # Rij
    Rij = np.swapaxes(np.reshape(np.cov(X), (N, D, N, D)), 1, 2)

    # Rw
    Rw = np.mean([Rij[i, i, :, :]
                  for i in range(0, N)], axis=0)
    # Rw_reg = (1 - gamma) * Rw + gamma * np.mean(eigh(Rw)[0]) * np.identity(Rw.shape[0])

    # Rb
    Rb = np.mean([Rij[i, j, :, :]
                  for i in range(0, N)
                  for j in range(0, N) if i != j], axis=0)

    # ISCs
    ISC = np.sort(np.diag(np.transpose(W) @ Rb @ W) / np.diag(np.transpose(W) @ Rw @ W))[::-1]

    # Scalp projections
    A = np.linalg.solve(Rw @ W, np.transpose(W) @ Rw @ W)  # a, b. 
    
    # ISC by subject
    print('by subject is calculating')
    ISC_bysubject = np.empty((D, N))

    for subj_k in range(0, N):
        Rw, Rb = 0, 0
        Rw = np.mean([Rw + 1 / (N - 1) * (Rij[subj_k, subj_k, :, :] + Rij[subj_l, subj_l, :, :])
                      for subj_l in range(0, N) if subj_k != subj_l], axis=0)
        Rb = np.mean([Rb + 1 / (N - 1) * (Rij[subj_k, subj_l, :, :] + Rij[subj_l, subj_k, :, :])
                      for subj_l in range(0, N) if subj_k != subj_l], axis=0)

        ISC_bysubject[:, subj_k] = np.diag(np.transpose(W) @ Rb @ W) / np.diag(np.transpose(W) @ Rw @ W)

    # ISC per second
    print('by persecond is calculating')
    ISC_persecond = np.empty((D, int(T / fs) ))
    window_i = 0

    for t in range(0, T, fs):

        Xt = X[:, t:t+window_sec*fs] #[subj 1, subj 2........subj 10]
       
        # the covariance
        Rij = np.cov(Xt) #910, 910for all the subjects 
        # <----10 subjects---->
        #  [sub1, sub2... sub10 ] sub 1
        #  [sub1, sub2... sub10 ] sub 2
        #  [sub1, sub2... sub10 ] ..
        #  [sub1, sub2... sub10 ] ..
        #   [sub1, sub2... sub10 ] sub 10
        
        
        
        Rw = np.mean([Rij[i:i + D, i:i + D] # Correlation diagonally (itself)
                      for i in range(0, D * N, D)], axis=0) 
        
        Rb = np.mean([Rij[i:i + D, j:j + D]
                      for i in range(0, D * N, D)
                      for j in range(0, D * N, D) if i != j], axis=0) #Correlation with other subjects
        
        ISC_persecond[:, window_i] = np.diag(np.transpose(W) @ Rb @ W) / np.diag(np.transpose(W) @ Rw @ W)
        window_i += 1
        
    stop = default_timer()
    print(f'Elapsed time: {round(stop - start)} seconds.')

    return ISC, ISC_persecond, ISC_bysubject, A

In [8]:
dic = dict()

dic['condition1'] = np.append(sub1_raw.get_data()[:91,516:85516].reshape(1,91,85000),sub2_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)

#for i in range(2,10):
#    dic['condition1'] = np.append(dic['condition1'],sub[i].get_data()[:,:86030].reshape(1,92,86030) ,axis=0)

dic['condition1'] = np.append(dic['condition1'],sub3_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub4_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub5_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub6_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub7_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub8_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub9_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)
dic['condition1'] = np.append(dic['condition1'],sub10_raw.get_data()[:91,516:85516].reshape(1,91,85000) ,axis=0)    

In [9]:
[W,ISC] = train_cca(dic)
#np.shape( sub1_raw.get_data() )

isc_results = dict()
for cond_key, cond_values in dic.items():
    isc_results[str(cond_key)] = dict(zip(['ISC', 'ISC_persecond', 'ISC_bysubject', 'A'], apply_cca(cond_values, W, 500)))


train_cca - calculations started. There are 1 conditions
Condition has 10 subjects, 91 sensors and 85000 samples
Elapsed time: 3 seconds.
apply_cca - calculations started
by subject is calculating
by persecond is calculating
Elapsed time: 8 seconds.


In [10]:
#[29,33],[67,71],[77,83],[130,135],[155,164]]]

#np.max(valstest,axis=0)

#significance
indexes = np.hstack([np.arange(159*500-1000,159*500+1060)])#,np.arange(67*500,72*500),np.arange(77*500,84*500),np.arange(130*500,136*500),np.arange(155*500,165*500)])
indexes2 = np.hstack([np.arange(100*500-1000,100*500+1060)])
#np.where(isc_results['condition1']['ISC_persecond'][0] == isc_results['condition1']['ISC_persecond'][0].max())

In [11]:
#isc_results['condition1']['A'][0]
#sub1_trial = isc_results['condition1']['A'][0] @ sub1_raw.get_data()
import tqdm
#sub1_trial = np.zeros([91,85001])
#a = [sub1_trial[:,i] (sub1_raw.get_data()[:,i]*isc_results['condition1']['A'][0]) for i in tqdm.tqdm(range(86040))]
 #sub1_trial[:,i] = (sub1_raw.get_data()[:,i]*isc_results['condition1']['A'][0])   

def element_wise_multi(epochs):
    subjects = isc_results['condition1']['A'][0].T @ epochs
    return subjects

def spatial_filter(epochs):
    subjects = np.multiply(epochs.T,isc_results['condition1']['A'][0])
    print(np.shape(subjects))
    subjects_final = np.reshape(subjects,[1,91,2060])
    return subjects_final
#np.shape(element_wise_multi(epochs1_ISC_ts.get_data()[:,:,indexes]))

#np.shape(isc_results['condition1']['A'][0],sub1_raw.get_data())

In [12]:
epochs1_ISC_ts = epochs(sub1_raw,sub1_events)
epochs2_ISC_ts = epochs(sub2_raw,sub2_events)
epochs3_ISC_ts = epochs(sub3_raw,sub3_events)
epochs4_ISC_ts = epochs(sub4_raw,sub4_events)
epochs5_ISC_ts = epochs(sub5_raw,sub5_events)
epochs6_ISC_ts = epochs(sub6_raw,sub6_events)
epochs7_ISC_ts = epochs(sub7_raw,sub7_events)
epochs8_ISC_ts = epochs(sub8_raw,sub8_events)
epochs9_ISC_ts = epochs(sub9_raw,sub9_events)
epochs10_ISC_ts = epochs(sub10_raw,sub10_events)


Not setting metadata
Not setting metadata
3 matching events found
Setting baseline interval to [0.0, 170.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 3 events and 85001 original time points ...
1 bad epochs dropped
Not setting metadata
Not setting metadata
3 matching events found
Setting baseline interval to [0.0, 170.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 3 events and 85001 original time points ...
1 bad epochs dropped
Not setting metadata
Not setting metadata
3 matching events found
Setting baseline interval to [0.0, 170.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 3 events and 85001 original time points ...
1 bad epochs dropped
Not setting metadata
Not setting metadata
3 matching events found
Setting baseline interval to [0.0, 170.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 3 eve

In [13]:
def get_raw_epochs_data(epochs):

    info = mne.create_info(sub1_raw.info['ch_names'],sfreq=500,ch_types = 'eeg')

    raw = mne.io.RawArray(epochs.reshape(91,2060),info)
    ep = mne.EpochsArray(epochs.reshape(1,91,2060),info)
    raw.set_montage('GSN-HydroCel-129')
    return info, raw, ep

def get_raw_epochs_data_elem(epochs):

    info = mne.create_info(['E1'],sfreq=500,ch_types = 'eeg')

    raw = mne.io.RawArray(epochs.reshape(1,2060),info)
    ep = mne.EpochsArray(epochs.reshape(1,1,2060),info)
    raw.set_montage('GSN-HydroCel-129')
    return info, raw, ep

In [14]:
from mne.datasets import fetch_fsaverage

import os.path as op
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)


subject = 'fsaverage' # Subject ID for the MRI-head transformation
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
source_space = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif') 

bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

0 files missing from root.txt in /homes/v20subra/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /homes/v20subra/mne_data/MNE-fsaverage-data/fsaverage


In [15]:
from mne.minimum_norm import make_inverse_operator, apply_inverse, apply_inverse_epochs

fwd_model = mne.make_forward_solution(sub1_raw.info, trans=trans, src=source_space, bem=bem, eeg=True, mindist=5.0)


Source space          : /homes/v20subra/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /homes/v20subra/.local/lib/python3.8/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /homes/v20subra/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /homes/v20subra/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  91 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BE

In [16]:
method = "eLORETA"
snr = 3.
lambda2 = 1. / snr ** 2

# epochs['30'].average() = Averaged evoked response for the event 30
#stc = apply_inverse_epochs(epochs_resampled, inverse_operator, lambda2,
 #                            method=method, pick_ori=None, verbose=True)
#    return stc

In [17]:
def comp_source_psd(raw):
 
    noise_cov = mne.compute_raw_covariance(raw)
    inverse_operator = mne.minimum_norm.make_inverse_operator(
        raw.info, forward=fwd_model, noise_cov=noise_cov, verbose=True)

    stc_psd, sensor_psd = mne.minimum_norm.compute_source_psd(
        raw, inverse_operator, lambda2=lambda2,method=method,fmax=40,
        dB=False, return_sensor=True, verbose=True)
    


    freq_bands = dict(
        delta=(2, 4), theta=(5, 7), alpha=(8, 13), beta=(15, 29), gamma=(30, 40))
    topos = dict(vv=dict(), opm=dict())
    stcs_dict = dict(vv=dict(), opm=dict())
    
    
    topo_norm = sensor_psd.data.sum(axis=1, keepdims=True)
    stc_norm = stc_psd.sum() 

    for band, limits in freq_bands.items():
        data = sensor_psd.copy().crop(*limits).data.sum(axis=1, keepdims=True)
        topos[band] = mne.EvokedArray(
            100 * data / topo_norm, sensor_psd.info)
        stcs_dict[band] = \
            100 * stc_psd.copy().crop(*limits).sum() / stc_norm.data
    
    return stcs_dict

In [18]:

epochs1_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs1_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs2_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs2_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs3_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs3_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs4_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs4_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs5_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs5_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs6_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs6_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs7_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs7_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs8_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs8_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs9_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs9_ISC_ts.get_data()[:,:,indexes], [91,2060]))
epochs10_ISC_ts_filtered_high = spatial_filter(np.reshape( epochs10_ISC_ts.get_data()[:,:,indexes], [91,2060]))
#(epochs10_ISC_ts.get_data()[:,:,indexes])




info1,raw1,ep1 = get_raw_epochs_data(epochs1_ISC_ts_filtered_high)
info2,raw2,ep2 = get_raw_epochs_data(epochs2_ISC_ts_filtered_high)
info3,raw3,ep3 = get_raw_epochs_data(epochs3_ISC_ts_filtered_high)
info4,raw4,ep4 = get_raw_epochs_data(epochs4_ISC_ts_filtered_high)
info5,raw5,ep5 = get_raw_epochs_data(epochs5_ISC_ts_filtered_high)
info6,raw6,ep6 = get_raw_epochs_data(epochs6_ISC_ts_filtered_high)
info7,raw7,ep7 = get_raw_epochs_data(epochs7_ISC_ts_filtered_high)
info8,raw8,ep8 = get_raw_epochs_data(epochs8_ISC_ts_filtered_high)
info9,raw9,ep9 = get_raw_epochs_data(epochs9_ISC_ts_filtered_high)
info10,raw10,ep10 = get_raw_epochs_data(epochs10_ISC_ts_filtered_high)



(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
   

In [19]:

# for the top and bottom 10%ile calculation for the source estimation of non-spatially filtered EEG
stc1_high = comp_source_psd(raw1)
stc2_high = comp_source_psd(raw2)
stc3_high = comp_source_psd(raw3)
stc4_high = comp_source_psd(raw4)
stc5_high = comp_source_psd(raw5)
stc6_high = comp_source_psd(raw6)
stc7_high = comp_source_psd(raw7)
stc8_high = comp_source_psd(raw8)
stc9_high = comp_source_psd(raw9)
stc10_high = comp_source_psd(raw10)

Using up to 20 segments
Number of samples used : 2000
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 91 channels.
    91 out of 91 channels remain after picking
Selected 91 channels
Creating the depth weighting matrix...
    91 EEG channels
    limit = 20485/20484 = 2.194331
    scale = 146509 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 4.1e-05 (2.2e-16 eps * 91 dim * 2e+09  max singular value)
    Estimated rank (eeg): 91
    EEG: rank 91 computed from 91 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Comput

<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(
<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(


    largest singular value = 5.54879
    scaling factor to adjust the trace = 1.8159e+11 (nchan = 91 nzero = 0)
Not setting metadata
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Considering frequencies 0 ... 40 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 91 (0 small eigenvalues omitted)
    Computing optimized source covariance (eLORETA)...
        Using independent orientation weights
        Fitting up to 20 iterations (this make take a while)...
        Converged on iteration 8 (7.3e-07 < 1e-06)
        Updating inverse with weighted eigen leads
[done]
Picked 91 channels from the data
Computing inverse...
    Eigenleads already weighted ... 
Reducing data rank 91 -> 91
Using hann windowing on at mos

  0%|          |  : 0/2 [00:00<?,       ?it/s]

Using up to 20 segments
Number of samples used : 2000
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 91 channels.
    91 out of 91 channels remain after picking


...

Selected 91 channels
Creating the depth weighting matrix...
    91 EEG channels
    limit = 20485/20484 = 2.194331
    scale = 146509 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 0.00018 (2.2e-16 eps * 91 dim * 8.7e+09  max singular value)
    Estimated rank (eeg): 91
    EEG: rank 91 computed from 91 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.


<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(
<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(


    largest singular value = 6.26896
    scaling factor to adjust the trace = 7.59786e+10 (nchan = 91 nzero = 0)
Not setting metadata
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Considering frequencies 0 ... 40 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 91 (0 small eigenvalues omitted)
    Computing optimized source covariance (eLORETA)...
        Using independent orientation weights
        Fitting up to 20 iterations (this make take a while)...
        Converged on iteration 9 (3.7e-07 < 1e-06)
        Updating inverse with weighted eigen leads
[done]
Picked 91 channels from the data
Computing inverse...
    Eigenleads already weighted ... 
Reducing data rank 91 -> 91
Using hann windowing on at mo

  0%|          |  : 0/2 [00:00<?,       ?it/s]

<ipython-input-17-555f416cc34f>:23: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (39.7949 sec)
  data = sensor_psd.copy().crop(*limits).data.sum(axis=1, keepdims=True)


In [20]:
#Low
epochs1_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs1_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs2_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs2_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs3_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs3_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs4_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs4_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs5_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs5_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs6_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs6_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs7_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs7_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs8_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs8_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs9_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs9_ISC_ts.get_data()[:,:,indexes2], [91,2060]))
epochs10_ISC_ts_filtered_low = spatial_filter(np.reshape( epochs10_ISC_ts.get_data()[:,:,indexes2], [91,2060]))





info1_low,raw1_low,ep1_low = get_raw_epochs_data(epochs1_ISC_ts_filtered_low)
info2_low,raw2_low,ep2_low = get_raw_epochs_data(epochs2_ISC_ts_filtered_low)
info3_low,raw3_low,ep3_low = get_raw_epochs_data(epochs3_ISC_ts_filtered_low)
info4_low,raw4_low,ep4_low = get_raw_epochs_data(epochs4_ISC_ts_filtered_low)
info5_low,raw5_low,ep5_low = get_raw_epochs_data(epochs5_ISC_ts_filtered_low)
info6_low,raw6_low,ep6_low = get_raw_epochs_data(epochs6_ISC_ts_filtered_low)
info7_low,raw7_low,ep7_low = get_raw_epochs_data(epochs7_ISC_ts_filtered_low)
info8_low,raw8_low,ep8_low = get_raw_epochs_data(epochs8_ISC_ts_filtered_low)
info9_low,raw9_low,ep9_low = get_raw_epochs_data(epochs9_ISC_ts_filtered_low)
info10_low,raw10_low,ep10_low = get_raw_epochs_data(epochs10_ISC_ts_filtered_low)




(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
(2060, 91)
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
    Range : 0 ... 2059 =      0.000 ...     4.118 secs
Ready.
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=91, n_times=2060
   

In [21]:
stc1_low = comp_source_psd(raw1_low)
stc2_low = comp_source_psd(raw2_low)
stc3_low = comp_source_psd(raw3_low)
stc4_low = comp_source_psd(raw4_low)
stc5_low = comp_source_psd(raw5_low)
stc6_low = comp_source_psd(raw6_low)
stc7_low = comp_source_psd(raw7_low)
stc8_low = comp_source_psd(raw8_low)
stc9_low = comp_source_psd(raw9_low)
stc10_low = comp_source_psd(raw10_low)

Using up to 20 segments
Number of samples used : 2000
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 91 channels.
    91 out of 91 channels remain after picking
Selected 91 channels
Creating the depth weighting matrix...
    91 EEG channels
    limit = 20485/20484 = 2.194331
    scale = 146509 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 4.3e-05 (2.2e-16 eps * 91 dim * 2.1e+09  max singular value)
    Estimated rank (eeg): 91
    EEG: rank 91 computed from 91 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Comp

<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(
<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(


    largest singular value = 6.06953
    scaling factor to adjust the trace = 2.08079e+11 (nchan = 91 nzero = 0)
Not setting metadata
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Considering frequencies 0 ... 40 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 91 (0 small eigenvalues omitted)
    Computing optimized source covariance (eLORETA)...
        Using independent orientation weights
        Fitting up to 20 iterations (this make take a while)...
        Converged on iteration 9 (3.3e-07 < 1e-06)
        Updating inverse with weighted eigen leads
[done]
Picked 91 channels from the data
Computing inverse...
    Eigenleads already weighted ... 
Reducing data rank 91 -> 91
Using hann windowing on at mo

  0%|          |  : 0/2 [00:00<?,       ?it/s]

Using up to 20 segments
Number of samples used : 2000
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 91 channels.
    91 out of 91 channels remain after picking


...

Selected 91 channels
Creating the depth weighting matrix...
    91 EEG channels
    limit = 20485/20484 = 2.194331
    scale = 146509 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 0.00042 (2.2e-16 eps * 91 dim * 2.1e+10  max singular value)
    Estimated rank (eeg): 91
    EEG: rank 91 computed from 91 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.


<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(
<ipython-input-17-555f416cc34f>:4: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(


    largest singular value = 6.04451
    scaling factor to adjust the trace = 5.94789e+10 (nchan = 91 nzero = 0)
Not setting metadata
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Considering frequencies 0 ... 40 Hz
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 91 (0 small eigenvalues omitted)
    Computing optimized source covariance (eLORETA)...
        Using independent orientation weights
        Fitting up to 20 iterations (this make take a while)...
        Converged on iteration 9 (3e-07 < 1e-06)
        Updating inverse with weighted eigen leads
[done]
Picked 91 channels from the data
Computing inverse...
    Eigenleads already weighted ... 
Reducing data rank 91 -> 91
Using hann windowing on at most

  0%|          |  : 0/2 [00:00<?,       ?it/s]

<ipython-input-17-555f416cc34f>:23: RuntimeWarning: tmax is not in Evoked time interval. tmax is set to evoked.tmax (39.7949 sec)
  data = sensor_psd.copy().crop(*limits).data.sum(axis=1, keepdims=True)


In [22]:
def high_minus_low(high,low,band):
    return high[band] - low[band]

def difference_and_t_test(band):
    alpha_difference_1 = high_minus_low(stc1_high,stc1_low,band)
    alpha_difference_2 = high_minus_low(stc2_high,stc2_low,band)
    alpha_difference_3 = high_minus_low(stc3_high,stc3_low,band)
    alpha_difference_4 = high_minus_low(stc4_high,stc4_low,band)
    alpha_difference_5 = high_minus_low(stc5_high,stc5_low,band)
    alpha_difference_6 = high_minus_low(stc6_high,stc6_low,band)
    alpha_difference_7 = high_minus_low(stc7_high,stc7_low,band)
    alpha_difference_8 = high_minus_low(stc8_high,stc8_low,band)
    alpha_difference_9 = high_minus_low(stc9_high,stc9_low,band)
    alpha_difference_10 = high_minus_low(stc10_high,stc10_low,band)
    averaged_alpha =(alpha_difference_1+alpha_difference_2+alpha_difference_3+alpha_difference_4+alpha_difference_5\
          +alpha_difference_6+alpha_difference_7+alpha_difference_8+alpha_difference_9+alpha_difference_10)/10
    a,b,c = mne.stats.permutation_t_test(averaged_alpha.data,n_permutations=1000)
    alpha_stacked = (np.hstack((alpha_difference_1.data,alpha_difference_2.data,\
                   alpha_difference_3.data,alpha_difference_4.data,\
                   alpha_difference_5.data,alpha_difference_6.data,\
                   alpha_difference_7.data,alpha_difference_8.data,\
                   alpha_difference_9.data,alpha_difference_10.data)))
    return averaged_alpha,alpha_stacked,b

In [23]:
averaged,to_test,b = difference_and_t_test('delta')


Permuting 999 times...


In [28]:
indices_for_slicing_beyond_percentiles = np.where( np.logical_or( (averaged.data <np.percentile(averaged.data,10)), (averaged.data>np.percentile(averaged.data,90))))

averaged.data[list(set(list(range(20484)))-set(indices_for_slicing_beyond_percentiles[0]))] =0

In [25]:
averaged

<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : 3051.7578125 (ms), tmax : 3051.7578125 (ms), tstep : 2197.265625 (ms), data shape : (20484, 1), ~320 kB>

In [ ]:
from nilearn import datasets
fsaverage = datasets.fetch_surf_fsaverage()
from nilearn import plotting

view= plotting.view_surf(fsaverage.infl_right,averaged.data[10242:],cmap='seismic')


In [ ]:
plot_nilearn(0,10242,'Alpha band')


In [25]:
import brainspace.mesh
mesh = brainspace.mesh.mesh_io.read_surface('S4B2/brainnotation/tpl-fsaverage_den-10k_hemi-L_pial.surf.gii')
mesh2 = brainspace.mesh.mesh_io.read_surface('S4B2/brainnotation/tpl-fsaverage_den-10k_hemi-R_pial.surf.gii')


In [42]:
from surfplot import Plot
%matplotlib qt
%gui qt
p = Plot(mesh,mesh2,zoom=1.2, views='lateral')
p.add_layer(averaged.data, cmap='seismic')
fig = p.build()
plt.title('plt.title() has worked !!')
fig.show()


In [49]:
averaged,to_test,b = difference_and_t_test('theta')
indices_for_slicing_beyond_percentiles = np.where( np.logical_or( (averaged.data <np.percentile(averaged.data,10)), (averaged.data>np.percentile(averaged.data,90))))

averaged.data[list(set(list(range(20484)))-set(indices_for_slicing_beyond_percentiles[0]))] =0
p = Plot(mesh,mesh2,zoom=1.2, views='lateral')
p.add_layer(averaged.data, cmap='seismic')
fig = p.build()

fig.show()


Permuting 999 times...


In [47]:
def surf_plot(band):
    averaged,to_test,b = difference_and_t_test(band)
    indices_for_slicing_beyond_percentiles = np.where( np.logical_or( (averaged.data <np.percentile(averaged.data,10)), (averaged.data>np.percentile(averaged.data,90))))

    averaged.data[list(set(list(range(20484)))-set(indices_for_slicing_beyond_percentiles[0]))] =0
    p = Plot(mesh,mesh2,zoom=1.2, views='lateral')
    p.add_layer(averaged.data, cmap='seismic')
    fig = p.build()
    plt.title('%s band'%band)
    fig.show()


In [48]:
surf_plot('delta')
surf_plot('theta')
surf_plot('alpha')
surf_plot('beta')
surf_plot('gamma')

Permuting 999 times...
Permuting 999 times...
Permuting 999 times...
Permuting 999 times...
Permuting 999 times...


In [86]:
cd

/homes/v20subra


In [99]:
band = ['theta','alpha','beta','gamma','delta']

l = difference_and_t_test(band[1])
plt.xlabel('p-value = {}'.format(l[2]))
print(l[0])
plt.boxplot(l[0].data)
plt.title('{} band (source level)'.format(band[1]))
#plt.savefig('{} band(source level).jpg'.format(band[0]))
#plt.close()

Permuting 999 times...
<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : 10620.1171875 (ms), tmax : 10620.1171875 (ms), tstep : 5126.953125 (ms), data shape : (20484, 1), ~320 kB>


Text(0.5, 1.0, 'alpha band (source level)')

In [5]:
import scipy
mat = scipy.io.loadmat('S4B2/subject_list.mat')

In [6]:
#pd.DataFrame(mat['good_EEG'])
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'good_EEG', 'not_bad_EEG', 'ratingsAuto'])

In [7]:
import numpy as np
a = np.random.randn(10,100,2)


In [74]:
#C = len(data.keys())
#print(f'train_cca - calculations started. There are {C} conditions')

from scipy.linalg import eigh


gamma = 0.1
Rw, Rb = 0, 0
#for cond in data.values():
N, D, T, = a.shape
print(f'Condition has {N} subjects, {D} sensors and {T} samples')
cond = a.reshape(D * N, T)

        # Rij
Rij = np.swapaxes(np.reshape(np.cov(cond), (N, D, N, D)), 1, 2)
        
        # Rw
Rw = Rw + np.mean([Rij[i, i, :, :]
                           for i in range(0, N)], axis=0)
        
        # Rb
Rb = Rb + np.mean([Rij[i, j, :, :]
                           for i in range(0, N)
                           for j in range(0, N) if i != j], axis=0)
covariance = np.cov(cond)
    # Divide by number of condition
C=1
Rw, Rb = Rw/C, Rb/C

    # Regularization
Rw_reg = (1 - gamma) * Rw + gamma * np.mean(eigh(Rw)[0]) * np.identity(Rw.shape[0])

    # ISCs and Ws
[ISC, W] = eigh(Rb, Rw_reg) #Eigen values, W matrix
    
    # Make descending order
ISC, W = ISC[::-1], W[:, ::-1] 
    #print(ISC[0])
#stop = default_timer()

#print(f'Elapsed time: {round(stop - start)} seconds.')
    #return W, ISC



Condition has 10 subjects, 100 sensors and 2 samples


In [75]:
W

array([[ 0.02585155, -0.19231059,  0.31268809, ..., -0.00220239,
        -0.03220372,  0.02940899],
       [-0.01616133, -0.07371589, -0.00110485, ..., -0.04106293,
         0.03372758, -0.02242619],
       [ 0.00330279,  0.05193175,  0.40651712, ..., -0.03326729,
        -0.05066275, -0.01251434],
       ...,
       [ 0.00793201,  0.13764298, -0.0111789 , ..., -0.03353268,
        -0.01360116, -0.02109327],
       [ 0.00183948,  0.30295573,  0.01314225, ...,  0.01677605,
        -0.00705577,  0.01947321],
       [ 0.07414329,  0.45278495, -0.53812491, ..., -0.02463864,
        -0.03179194,  0.03524768]])

In [67]:
Rij[1,1,:,:]

array([[0.92829162]])